In [2]:
# Import necessary libraries and modules
import numpy as np
from numpy import pi, random
from math import sqrt, atan2, cos, sin, pow
from qiskit import *
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit.extensions import *

In [3]:
# Define desired states
# Alice Input states
theta_A_1 = pi/3
theta_A_2 = pi/5
a0 = cos(theta_A_1)
a1 = sin(theta_A_1)
a2 = cos(theta_A_2)
a3 = sin(theta_A_2)

# Bob Input states
theta_B_1 = pi/6
theta_B_2 = pi/4
b0 = cos(theta_B_1)
b1 = sin(theta_B_1)
b2 = cos(theta_B_2)
b3 = sin(theta_B_2)

# Define Classica and Quantum bits
# Alice Side
# q_A[0]=|A1⟩, q_A[1]=|A2⟩
# q_a[0]=|a1⟩, q_a[1]=|a2⟩, q_a[2]=|a3⟩, q_a[3]=|a4⟩
q_A = QuantumRegister(2, 'q_A')
q_a = QuantumRegister(4, 'q_a')

# Bob Side
# q_B[0]=|B1⟩, q_B[1]=|B2⟩
# q_b[0]=|b1⟩, q_b[1]=|b2⟩, q_b[2]=|b3⟩, q_b[3]=|b4⟩
q_B = QuantumRegister(2, 'q_B')
q_b = QuantumRegister(4, 'q_b')

# Charlie Side
# q_C_A[0]=|CA1⟩, q_C_A[1]=|CA2⟩
# q_C_B[0]=|CB1⟩, q_C_B[1]=|CB2⟩
q_C_A = QuantumRegister(2, 'q_C_A')
q_C_B = QuantumRegister(2, 'q_C_B')

# First Hop
# q_h[0]=|h1⟩, q_h[1]=|h2⟩, q_h[2]=|h3⟩, q_h[3]=|h4⟩
# q_h[4]=|h'1⟩, q_h[5]=|h'2⟩, , q_h[6]=|h'3⟩, q_h[7]=|h'4⟩
q_h = QuantumRegister(8, 'q_h')

# Classic bits: 
# M[0]=|A1⟩, M[1]=|A2⟩, M[2]=|a1⟩, M[3]=|a2⟩
# M[4]=|B1⟩, M[5]=|B2⟩, M[6]=|b3⟩, M[7]=|b4⟩
# M[8]=|h1⟩, M[9]=|h2⟩, M[10]=|h3⟩, M[11]=|h4⟩
# M[12]=|h'1⟩, M[13]=|h'2⟩, M[14]=|h'3⟩, M[15]=|h'4⟩
# M[16]=|CA1⟩, M[17]=|CA2⟩, M[18]=|CB1⟩, M[19]=|CB2⟩
# M[20]=|a3⟩, M[21]=|a4⟩, M[22]=|b1⟩, M[23]=|b2⟩
M = ClassicalRegister(24,'m')

# Define Quantum Circuit
QC = QuantumCircuit(q_A,q_B,q_C_A,q_C_B,q_a,q_b,q_h,M)

# Quantum state initialization
QC.initialize([a0,a1],q_A[0])
QC.initialize([a2,a3],q_A[1])
QC.initialize([b0,b1],q_B[0])
QC.initialize([b2,b3],q_B[1])
QC.barrier()

# Prepare quantum channel
QC.h(q_a[0:4])
QC.h(q_b[0:4])
QC.cx(q_a[0],q_h[0])
QC.cx(q_a[1],q_h[2])
QC.cx(q_a[0],q_C_A[0])
QC.cx(q_a[1],q_C_A[1])
QC.cx(q_a[2],q_h[5])
QC.cx(q_a[3],q_h[7])

QC.cx(q_b[0],q_h[1])
QC.cx(q_b[1],q_h[3])
QC.cx(q_b[2],q_C_B[0])
QC.cx(q_b[3],q_C_B[1])
QC.cx(q_b[2],q_h[4])
QC.cx(q_b[3],q_h[6])

QC.barrier()


# BSM on Alice and Bob's qubits
# BSM on M[0]=q_A[0]=|A1⟩,  M[2]=q_a[0]=|a1⟩
# BSM on M[1]=q_A[1]=|A2⟩,  M[3]=q_a[1]=|a2⟩
QC.cx(q_A[0],q_a[0])
QC.cx(q_A[1],q_a[1])
QC.h(q_A[0])
QC.h(q_A[1])
QC.measure(q_A[0],M[0])
QC.measure(q_A[1],M[1])
QC.measure(q_a[0],M[2])
QC.measure(q_a[1],M[3])

# BSM on M[4]=q_B[0]=|B1⟩,  M[6]=q_b[2]=|b3⟩
# BSM on M[5]=q_B[1]=|B2⟩,  M[7]=q_b[3]=|b4⟩
QC.cx(q_B[0],q_b[2])
QC.cx(q_B[1],q_b[3])
QC.h(q_B[0])
QC.h(q_B[1])
QC.measure(q_B[0],M[4])
QC.measure(q_B[1],M[5])
QC.measure(q_b[2],M[6])
QC.measure(q_b[3],M[7])

QC.barrier()


# Apply required gates on Charlie's and Hop's qubits
# Charlie's qubits: q_C_A[0]=|CA1⟩, q_C_A[1]=|CA2⟩, q_C_B[0]=|CB1⟩ q_C_B[1]=|CB2⟩
# Hop's qubits:     q_h[0]=|h1⟩, q_h[2]=|h3⟩, q_h[4]=|h'1⟩, q_h[5]=|h'3⟩
# BSM Results: |A1a1⟩ and |A2a2⟩
QC.z(q_h[0]).c_if(M[0],1)
QC.x(q_h[0]).c_if(M[2],1)
QC.z(q_h[2]).c_if(M[1],1)
QC.x(q_h[2]).c_if(M[3],1)
QC.z(q_C_A[0]).c_if(M[0],1)
QC.x(q_C_A[0]).c_if(M[2],1)
QC.z(q_C_A[1]).c_if(M[1],1)
QC.x(q_C_A[1]).c_if(M[3],1)
# BSM Results: |B1b3⟩ and |B2b4⟩
QC.z(q_h[4]).c_if(M[4],1)
QC.x(q_h[4]).c_if(M[6],1)
QC.z(q_h[6]).c_if(M[5],1)
QC.x(q_h[6]).c_if(M[7],1)
QC.z(q_C_B[0]).c_if(M[4],1)
QC.x(q_C_B[0]).c_if(M[6],1)
QC.z(q_C_B[1]).c_if(M[5],1)
QC.x(q_C_B[1]).c_if(M[7],1)

QC.barrier()


# BSM on Hop qubits
# BSM on M[8]=q_h[0]=|h1⟩,  M[9]=q_h[1]=|h2⟩
# BSM on M[10]=q_h[2]=|h3⟩,  M[11]=q_h[3]=|h4⟩
QC.cx(q_h[0],q_h[1])
QC.cx(q_h[2],q_h[3])
QC.h(q_h[0])
QC.h(q_h[2])
QC.measure(q_h[0],M[8])
QC.measure(q_h[1],M[9])
QC.measure(q_h[2],M[10])
QC.measure(q_h[3],M[11])

# BSM on M[12]=q_h[4]=|h'1⟩,  M[13]=q_h[3]=|h'2⟩
# BSM on M[12]=q_h[6]=|h'3⟩,  M[14]=q_h[3]=|h'4⟩
QC.cx(q_h[4],q_h[5])
QC.cx(q_h[6],q_h[7])
QC.h(q_h[4])
QC.h(q_h[6])
QC.measure(q_h[4],M[12])
QC.measure(q_h[5],M[13])
QC.measure(q_h[6],M[14])
QC.measure(q_h[7],M[15])

QC.barrier()


# Apply required gates on Alice's and Bob's qubits
# q_a[2]=|a3⟩, q_a[3]=|a4⟩, q_b[0]=|b1⟩, q_b[1]=|b2⟩
# BSM Results: |h1h2⟩ and |h3h4⟩
QC.z(q_b[0]).c_if(M[8],1)
QC.x(q_b[0]).c_if(M[9],1)
QC.z(q_b[1]).c_if(M[10],1)
QC.x(q_b[1]).c_if(M[11],1)

# BSM Results: |h'1h'2⟩ and |h'3h'4⟩
QC.z(q_a[2]).c_if(M[12],1)
QC.x(q_a[2]).c_if(M[13],1)
QC.z(q_a[3]).c_if(M[14],1)
QC.x(q_a[3]).c_if(M[15],1)

QC.barrier()

# Z-basis Measurement on Charlie's qubits
# M[16]=q_C_A[0]=|CA1⟩, M[17]=q_C_A[2]=|CA2⟩,  M[18]=q_C_B[0]=|CB1⟩,  M[19]=q_C_B[1]=|CB2⟩
QC.h(q_C_A[0])
QC.h(q_C_A[1])
QC.h(q_C_B[0])
QC.h(q_C_B[1])
QC.measure(q_C_A[0],M[16])
QC.measure(q_C_A[1],M[17])
QC.measure(q_C_B[0],M[18])
QC.measure(q_C_B[1],M[19])

QC.barrier()


# Apply required gates on Alice's and Bob's qubits
# q_a[2]=|a3⟩, q_a[3]=|a4⟩, q_b[0]=|b1⟩, q_b[1]=|b2⟩
# BSM Results: |CA1CA2⟩ and |CB1CB2⟩
QC.z(q_b[0]).c_if(M[16],1)
QC.z(q_b[1]).c_if(M[17],1)
QC.z(q_a[2]).c_if(M[18],1)
QC.z(q_a[3]).c_if(M[19],1)


QC.barrier()

# Z-basis measurement on Output qubits
# q_a[2]=|a3⟩, q_a[3]=|a4⟩, q_b[0]=|b1⟩, q_b[1]=|b2⟩
QC.measure(q_a[2],M[20])
QC.measure(q_a[3],M[21])
QC.measure(q_b[0],M[22])
QC.measure(q_b[1],M[23])


#QC.draw()

In [ ]:
# Set the number of shots for the simulation
m = 1000

# Choose the QasmSimulator for the simulation
simulator = QasmSimulator()

# Transpile the quantum circuit for execution on the simulator
circ = transpile(QC, simulator)

# Run the simulation with the specified number of shots
result = simulator.run(circ, shots=m).result()

# Obtain the measurement outcomes and their counts
counts = result.get_counts(circ)

# Display the histogram of measurement outcomes
# You can adjust the figure size using the second argument in plot_histogram
# [20, 6.5] specifies the width and height of the figure
plot_histogram(counts, [50, 6.5],title='Quantum System Measurement Results',color='skyblue')

In [5]:
# Initialize an array to store counts for different states
State_a3 = np.zeros([2])
State_a4 = np.zeros([2])
State_b1 = np.zeros([2])
State_b2 = np.zeros([2])

# Count occurrences of different states based on measurement outcomes
for state in counts:
    if state[0] == '0':
        State_b2[0] = State_b2[0] + counts[state]
    else:
        State_b2[1] = State_b2[1] + counts[state]
    
    if state[1] == '0':
        State_b1[0] = State_b1[0] + counts[state]
    else:
        State_b1[1] = State_b1[1] + counts[state]
    
    if state[2] == '0':
        State_a4[0] = State_a4[0] + counts[state]
    else:
        State_a4[1] = State_a4[1] + counts[state]
        
    if state[3] == '0':
        State_a3[0] = State_a3[0] + counts[state]
    else:
        State_a3[1] = State_a3[1] + counts[state]


counts_a3 = {'0': State_a3[0], '1': State_a3[1]}
counts_a4 = {'0': State_a4[0], '1': State_a4[1]}
counts_b1 = {'0': State_b1[0], '1': State_b1[1]}
counts_b2 = {'0': State_b2[0], '1': State_b2[1]}


In [4]:
# Display a histogram of measurement outcomes for Alice's qubit
print('The Bob input states:')
print('|φ_B_1> = β0|0> + β1|1>')
print('|φ_B_2> = β2|0> + β3|1>')
print('β0 = ', cos(theta_B_1))
print('β1 = ', sin(theta_B_1))
print('β2 = ', cos(theta_B_2))
print('β3 = ', sin(theta_B_2))
print('P_B1_|0⟩ = ', pow(cos(theta_B_1),2))
print('P_B1_|1⟩ = ', pow(sin(theta_B_1),2))
print('P_B2_|0⟩ = ', pow(cos(theta_B_2),2))
print('P_B2_|1⟩ = ', pow(sin(theta_B_2),2))

The Bob input states:
|φ_B_1> = β0|0> + β1|1>
|φ_B_2> = β2|0> + β3|1>
β0 =  0.8660254037844387
β1 =  0.49999999999999994
β2 =  0.7071067811865476
β3 =  0.7071067811865476
P_B1_|0⟩ =  0.7500000000000001
P_B1_|1⟩ =  0.24999999999999994
P_B2_|0⟩ =  0.5000000000000001
P_B2_|1⟩ =  0.5000000000000001


In [ ]:
plot_histogram(counts_a3, title='Alice Measurement Results: Teleported State B1',color='#FF0000')

In [ ]:
plot_histogram(counts_a4, title='Alice Measurement Results: Teleported State B2',color='#FF0000')

In [5]:
# Display a histogram of measurement outcomes for Bob's qubit
print('The Alice input states:')
print('|φ_A_1> = a0|0> + a1|1>')
print('|φ_A_2> = a2|0> + a3|1>')
print('a0 = ', cos(theta_A_1))
print('a1 = ', sin(theta_A_1))
print('a2 = ', cos(theta_A_2))
print('a3 = ', sin(theta_A_2))
print('P_A1_|0⟩ = ', pow(cos(theta_A_1),2))
print('P_A1_|1⟩ = ', pow(sin(theta_A_1),2))
print('P_A2_|0⟩ = ', pow(cos(theta_A_2),2))
print('P_A2_|1⟩ = ', pow(sin(theta_A_2),2))

The Alice input states:
|φ_A_1> = a0|0> + a1|1>
|φ_A_2> = a2|0> + a3|1>
a0 =  0.5000000000000001
a1 =  0.8660254037844386
a2 =  0.8090169943749475
a3 =  0.5877852522924731
P_A1_|0⟩ =  0.2500000000000001
P_A1_|1⟩ =  0.7499999999999999
P_A2_|0⟩ =  0.6545084971874737
P_A2_|1⟩ =  0.3454915028125263


In [ ]:
plot_histogram(counts_b1, title='Bob Measurement Results: Teleported State A1',color='#FF0000')

In [ ]:
plot_histogram(counts_b2, title='Bob Measurement Results: Teleported State A2',color='#FF0000')